# Example 12: KCube DC servo
This example shows a simple interactive movement based on ipywidgets

In [1]:
import os
import sys
import ctypes
import time
import ipywidgets as widgets
from ipywidgets import interact, interactive
#os.environ['PATH'] = "C:\\Program Files\\ThorLabs\\Kinesis" + ";" + os.environ['PATH']
os.add_dll_directory("C:\\Program Files\\ThorLabs\\Kinesis")  
sys.path.append(r"thorlabs-kinesis")

In [2]:
from thorlabs_kinesis.kcube_dc_device import kcube
from thorlabs_kinesis import kcube_dcservo as kcdc

In [3]:
if kcdc.TLI_BuildDeviceList() == 0:
    print("Device list built (no errors).")
serial_no = "27256231"
serialno = ctypes.c_char_p(bytes(serial_no, "utf-8"))

Device list built (no errors).


In [4]:
serialno = ctypes.c_char_p(bytes(serial_no, "utf-8"))
message_type = ctypes.wintypes.WORD()
message_id = ctypes.wintypes.WORD()
message_data = ctypes.wintypes.DWORD()

def move_to_pos(move_pos_mm):
    move_pos = int(move_pos_mm * kcube.steps_per_mm)
    motor_command = ctypes.c_int(move_pos)

    try:
        # Open Communication
        kcdc.CC_Open(serialno)
        kcdc.CC_StartPolling(serialno, ctypes.c_int(200))
        kcdc.CC_ClearMessageQueue(serialno)
        time.sleep(0.5)
                
        # Start Move Test
        w_move_state.value = False
        w_move_state.description = "moving"
        kcdc.CC_ClearMessageQueue(serialno)
        kcdc.CC_MoveToPosition(serialno, motor_command)
        kcdc.CC_WaitForMessage(serialno, ctypes.byref(message_type), 
                               ctypes.byref(message_id), ctypes.byref(message_data))

        while (int(message_type.value) != 2) or (int(message_id.value) != 1):
            kcdc.CC_WaitForMessage(serialno, ctypes.byref(message_type), 
                                   ctypes.byref(message_id), ctypes.byref(message_data))
            kcdc.CC_RequestPosition(serialno)
            w_out.value = kcdc.CC_GetPosition(serialno)/kcube.steps_per_mm
        w_move_state.value = True
        w_move_state.description = "finished"

    finally:
        # # Close Communication
        kcdc.CC_StopPolling(serialno)
        kcdc.CC_Close(serialno)

In [5]:
# Open Communication
current_position = 0

try:
    kcdc.CC_Open(serialno)
    kcdc.CC_StartPolling(serialno, ctypes.c_int(200))
    kcdc.CC_ClearMessageQueue(serialno)
    time.sleep(1)
    #Get Motor Position
    current_position = kcdc.CC_GetPosition(serialno)/kcube.steps_per_mm
    current_position = float(rf"{current_position:7.4f}")
finally:
    kcdc.CC_Close(serialno)
    
w_pos_in = widgets.FloatText(min=0.00, max=24.5, value=current_position, step=0.01, description="Set Position [mm]")
w_move_btn = widgets.Button(description='Move')

w_out = widgets.FloatText(value=current_position, description="Current Position [mm]", disabled = True)
w_move_state = widgets.Valid(
    value=True,
    description='finished!',
)

widgets.interact_manual(move_to_pos, move_pos_mm=w_pos_in )
display(w_out)
display(w_move_state)

interactive(children=(FloatText(value=10.023, description='Set Position [mm]', step=0.01), Button(description=…

FloatText(value=10.023, description='Current Position [mm]', disabled=True)

Valid(value=True, description='finished!')